In [1]:
import PySimpleGUI as sg

# Helper function to calculate total net worth
def calculate_net_worth(assets):
    total_net_worth = sum(asset['price'] * asset['quantity'] for asset in assets)
    individual_values = [(asset['price'] * asset['quantity']) for asset in assets]
    return total_net_worth, individual_values

# Helper function to calculate the next month's passive income
def calculate_passive_income(assets):
    monthly_income = 0
    for asset in assets:
        monthly_return = asset['price'] * asset['quantity'] * (asset['yearly_return'] / 100) / 12
        monthly_income += monthly_return
        if asset['reinvest_profit']:
            asset['price'] += monthly_return  # Reinvest the monthly return back into the asset
    return monthly_income

# Define the layout for the GUI
layout = [
    [sg.Text("Asset Manager", size=(30, 1), font=("Helvetica", 25), text_color='blue')],
    [sg.Text("Add your assets and details below")],
    [sg.Text("Asset Name"), sg.InputText(key='asset_name')],
    [sg.Text("Price"), sg.InputText(key='price')],
    [sg.Text("Quantity"), sg.InputText(key='quantity')],
    [sg.Text("Yearly Return (%)"), sg.InputText(key='yearly_return')],
    [sg.Checkbox("Reinvest Profit", key='reinvest_profit')],
    [sg.Button("Add Asset"), sg.Button("Remove Asset"), sg.Button("Clear List")],
    [sg.Listbox(values=[], size=(50, 6), key='asset_list')],
    [sg.Text("Total Net Worth:"), sg.Text('0', size=(20, 1), key='total_net_worth')],
    [sg.Text("Budget:"), sg.InputText('10000', key='budget')],
    [sg.ProgressBar(max_value=100, orientation='h', size=(20, 20), key='progress_bar')],
    [sg.Text("Estimated Passive Income Next Month:"), sg.Text('$0', size=(20, 1), key='passive_income')],
    [sg.Exit()]
]

# Create the window
window = sg.Window("Asset Manager", layout, background_color="darkgrey")

assets = []

# Event loop to process events and inputs
while True:
    event, values = window.read()

    # If user closes window or clicks 'Exit', end the program
    if event in (sg.WIN_CLOSED, 'Exit'):
        break

    if event in ["Add Asset", "Remove Asset", "Clear List"]:
        # Asset operations
        if event == "Add Asset":
            try:
                if not values['asset_name'] or not values['price'] or not values['quantity'] or not values['yearly_return']:
                    raise ValueError("Please fill in all fields.")
                asset_details = {
                    'name': values['asset_name'],
                    'price': float(values['price']),
                    'quantity': float(values['quantity']),
                    'yearly_return': float(values['yearly_return']),
                    'reinvest_profit': float(values['reinvest_profit'])
                }
                assets.append(asset_details)
            except ValueError as e:
                sg.popup_error(str(e))
        
        if event == "Remove Asset":
            if values['asset_list']:  # Check if an item is selected
                selected_asset = values['asset_list'][0]
                # Match the 'name' of the asset to remove with the selected list item
                assets = [asset for asset in assets if asset['name'] != selected_asset[0]]

       

        if event == "Clear List":
            assets.clear()

        visible_properties = ['name', 'price', 'quantity']  # Įtraukite tik tas savybes, kurias norite matyti
        # Update the Listbox with the new asset list
        window['asset_list'].update(values=[f"{a['name']:<20}{a['price']:<15} * {a['quantity']:^15} = ${a['price'] * a['quantity']:.2f}" for a in assets])
       
        # Calculate and display the estimated passive income for the next month
        passive_income = calculate_passive_income(assets)
        window['passive_income'].update(f"${passive_income:.2f}")

        # Calculate and display total net worth
        total_net_worth, _ = calculate_net_worth(assets)
        formatted_total_net_worth = f"${total_net_worth:.2f}"
        window['total_net_worth'].update(formatted_total_net_worth)
        
        # Check if total net worth exceeds the target
        try:
            budget = float(values['budget']) 
        except ValueError:
            sg.popup_error("Please enter a valid number for net worth target.")
            continue  # Grįžtame į įvesties laukelį

        updated_net_worth, individual_values = calculate_net_worth(assets)

        if updated_net_worth < budget:
            progress = updated_net_worth / budget * 100
            window['progress_bar'].update(current_count=progress, max=100)
            sg.popup("You can keep buying")
        else:
            sg.popup_error("You do not have so much money")
window.close()